In [ ]:
import os
import sys
import glob
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import matplotlib as mpl
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import matplotlib.font_manager as fm

from mpl_toolkits.mplot3d import Axes3D
from IPython.core.interactiveshell import InteractiveShell
!pip install factor_analyzer
from sklearn.impute import SimpleImputer
from factor_analyzer import FactorAnalyzer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from factor_analyzer.factor_analyzer import calculate_kmo
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
#各种设置
warnings.simplefilter("ignore")
__file__ = './datasets/src'
sys.path.append(os.path.realpath(__file__))
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_columns = 1000
myfont = mpl.font_manager.FontProperties(fname="/Library/Fonts/SimHei.ttf")  #"/Library/Fonts/Songti.ttc") 字体设置
mpl.rcParams["axes.unicode_minus"] = False

mpl.style.use('ggplot')
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False    # 用来正常显示负号,#有中文出现的情况，需要u'内容'

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
loadpath = "/content/drive/My Drive/Colab Notebooks/XGB用户画像"
print(os.chdir(loadpath))
print(os.listdir(loadpath))#目前路径
# import zipfile
# # 传入压缩文件zfile.zip获取相关信息
# zip_file = zipfile.ZipFile(loadpath+'/datasets.zip')
# # 获取压缩文件中的内容
# f_content = zip_file.namelist()
# f_content
# #解压缩到主目录
# zip_extract = zip_file.extractall(loadpath)

In [ ]:
if not os.path.exists('./datasets/src/temp/'):
    os.mkdir(os.getcwd()+'/datasets/src/temp/')
if not os.path.exists('./datasets/src/temp/pictures'):
    os.mkdir(os.getcwd()+'/datasets/src/temp/pictures')

In [ ]:
data = pd.read_csv('./datasets/data/LoanStat.csv')
data.head(5)

In [ ]:
print('The shape of data: ',data.shape)
print('==========================================================================================')
print('每列对应的null的个数:',data.isnull().sum())
print('==========================================================================================')
null_stat = data.isnull().sum()/len(data)
print('缺失值占比:',null_stat)
print('==========================================================================================')
null_col = null_stat[null_stat>0.8].index.tolist()
print('缺失值占比在80%以上的列:')
null_col

In [ ]:
rename_dic = {'id':'ID',
              'member_id':'用户ID',
              'loan_amnt':'贷款金额',
              'funded_amnt':'承诺放贷金额（贷款人）',
              'funded_amnt_inv':'承诺放贷金额（贷款）',
              'term':'贷款期数',
              'int_rate':'收入验证',
              'installment':'每月还款',
              'grade':'贷款等级',
              'inq_last_12m':'过去12个月的信用查询次数',
              'sub_grade':'贷款子等级',
              'emp_title':'职称',
              'emp_length':'就业年限',
              'home_ownership':'房屋所有权状态',
              'annual_inc':'年收入',
              'verification_status':'1',
              'issue_d':'贷款获得资助的年月',
              'loan_status':'贷款现状',
              'pymnt_plan':'是否实施还款计划',
              'desc':'贷款说明',
              'purpose':'贷款目的',
              'title':'贷款头衔',
              'zip_code':'邮政编码',
              'addr_state':'州简称',
              'dti':'收入负债比（总额）',
              'delinq_2yrs':'过去两年借款人逾期30天以上的事件数',
              'earliest_cr_line':'信贷额度开启的月份',
              'inq_last_6mths':'过去6个月的查询数目',
              'mths_since_last_delinq':'最后一次拖欠以来的月数',
              'mths_since_last_record':'上次公开记录以来的月数',
              'open_acc':'未结信用额度',
              'pub_rec':'贬损公共记录的数量',
              'revol_bal':'总信贷周转余额',
              'revol_util':'循环利用率',
              'total_acc':'当前信用额度的总数',
              'initial_list_status':'贷款的初始上市状态',
              'out_prncp':'剩余未偿还本金',
              'out_prncp_inv':'剩余未偿还本金（本人）',
              'total_pymnt':'迄今还款总额',
              'total_pymnt_inv':'迄今为止收到的由投资者资助的部分总金额',
              'total_rec_prncp':'迄今已收本金',
              'total_rec_int':'迄今已收利息',
              'total_rec_late_fee':'迄今已收滞纳金',
              'recoveries':'收取总回收费用',
              'collection_recovery_fee':'收取手续费',
              'last_pymnt_d':'上个月收到付款',
              'last_pymnt_amnt':'上次收到的总付款金额',
              'next_pymnt_d':'下一个预定付款日期',
              'last_credit_pull_d':'最近一个月提供信贷的日期',
              'collections_12_mths_ex_med':'除医疗收集外12个月的馆藏数量',
              'mths_since_last_major_derog':'最近90天或更差评级以来月份数',
              'policy_code':'产品是否公开',
              'application_type':'是否是单独申请',
              'annual_inc_joint':'合并借款人年度收入',
              'dti_joint':'收入负债比(月)',
              'verification_status_joint':'0',
              'acc_now_delinq':'借款人现在欠款的账户数量',
              'tot_coll_amt':'欠款总额',
              'tot_cur_bal':'所有帐户的当前总余额',
              'open_acc_6m':'过去6个月的未平仓交易数量',
              'open_il_12m':'过去12个月内开立的分期付款帐户数量',
              'open_il_24m':'过去24个月内开立的分期付款帐户数量',
              'mths_since_rcnt_il':'最近分期付款帐户开通以来的月份数',
              'total_bal_il':'所有分期付款帐户的当前总余额',
              'il_util':'所有安装帐户的总流量余额与信用额度的比率',
              'open_rv_12m':'过去12个月内开设的循环交易数量',
              'open_rv_24m':'过去24个月内开设的循环交易数量',
              'max_bal_bc':'所有周转账户的最大当前余额',
              'all_util':'所有交易均衡信贷限额',
              'total_rev_hi_lim':'总循环高信用/信用额度',
              'inq_fi':'个人理财咨询数量',
              'total_cu_tl':'融资交易数量',
              'total_bal_ex_mort':'除房贷之外的信用负债',
              'num_tl_30dpd':'30天内需结清的款项'}
print('筛选之后的重命名变量长度:',len(rename_dic))

In [ ]:
pd.options.display.max_columns = 100#只显示100列
# 只留下改名后的字段
data = data.rename(columns=rename_dic)[list(rename_dic.values())]
data = data.drop(['0', '1'], axis=1)

In [ ]:
data.head(5)

In [ ]:
# 将 'issue_d' str形式的时间日期转换
dt_series = pd.to_datetime(data['贷款获得资助的年月'])
data['贷款获得资助的年份'] = dt_series.dt.year
data['贷款获得资助的月份'] = dt_series.dt.month
# 数据分箱：根据收入 'annual_inc' 定义收入“低、中、高”
# <10万 --> low , 10万~20万 --> medium , >20 万 --> high
data['年收入层级'] = np.nan
data.loc[data['年收入'] <= 100000, '年收入层级'] = 'Low'
data.loc[(data['年收入'] > 100000) & (data['年收入'] <= 200000), '年收入层级'] = 'Medium'
data.loc[data['年收入'] > 200000, '年收入层级'] = 'High'

In [ ]:
# 字段贷款期数清洗
data['贷款期数'] = data['贷款期数'].map(lambda x: x.split(' months')[0]).astype(int)#把string转换成int月份

# 字段收入验证清洗
data['收入验证'] = data['收入验证'].map(lambda x: x.split('%')[0]).astype(float)

# 字段就业年限清洗
dic = {'< 1 year':1, '1 year':2, '2 years':3, '3 years':4, '4 years':5, '5 years':6,
       '6 years':7, '7 years':8, '8 years':9, '9 years':10,  '10+ years':11}
data['就业年限等级'] = data['就业年限'].map(dic)
data["就业年限"] = data["就业年限"].replace({'years':'','year':'',' ':'','<':'','\+':'','n/a':'0'}, regex = True).astype(float)

# 字段贷款等级/贷款子等级清洗
dic = {'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7}
data['贷款等级_INT'] = data['贷款等级'].map(dic)

dic = dict(zip(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2',
                'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4',
                'E5', 'F1', 'F2', 'F3', 'F4', 'F5', 'G1', 'G2', 'G3', 'G4', 'G5'],
                range(data['贷款子等级'].nunique())))
data['贷款子等级'] = data['贷款子等级'].map(dic)

In [ ]:
# 字段循环利用率清洗
data['循环利用率'] = data['循环利用率'].astype(str).map(lambda x: x.split('%')[0]).astype(float)

# 字段房屋所有权状态清洗
data["房屋所有权状态等级"] = data["房屋所有权状态"].map({"MORTGAGE":6,"RENT":5,"OWN":4,"OTHER":3,"NONE":2,"ANY":1})

data = data.apply(pd.to_numeric, errors='ignore')

In [ ]:
# 根据贷款状态 'loan_status' 定义好坏贷款
data['贷款现状'].value_counts()

bad_loan = ["Charged Off",
            "Default",
            "Does not meet the credit policy. Status:Charged Off",
            "In Grace Period",
            "Late (16-30 days)",
            "Late (31-120 days)"]

def loan_condition(status):
    if status in bad_loan:
        return 'Bad Loan'
    else:
        return 'Good Loan'

data['贷款现状'] = data['贷款现状'].apply(loan_condition)
data['贷款现状标签'] = np.nan
data.loc[data['贷款现状'] == 'Good Loan', '贷款现状标签'] = 0  # Negative (Bad Loan)
data.loc[data['贷款现状'] == 'Bad Loan', '贷款现状标签'] = 1  # Positive (Good Loan)
data['贷款现状标签'] = data['贷款现状标签'].astype(int)

In [ ]:
for i in ['贷款等级', '贷款子等级', '职称', '贷款获得资助的年月', '贷款现状', '是否实施还款计划',
        '贷款目的', '贷款头衔', '邮政编码', '州简称', '贷款的初始上市状态', '上个月收到付款',
        '最近一个月提供信贷的日期', '是否是单独申请', '年收入层级']:
    if data[i].nunique()<=20:
        tmp = pd.get_dummies(data[i], prefix=i)
        data = pd.concat([data, tmp], axis=1)
        print('Processing:' + i)

In [ ]:
#数据去重
data = data.drop_duplicates(subset=['ID', '用户ID'], keep='first')
data = data.sort_values(by='贷款获得资助的年月', ascending=False)
df = data.drop_duplicates(subset=['用户ID'], keep='first')
df.columns = ['最后一次借款_'+i for i in df.columns]
df = df.rename(columns={'最后一次借款_ID':'ID', '最后一次借款_用户ID':'用户ID'}) 
print('用户样本的大小为：%d'%df.shape[0])
print('贷款案件样本的大小为：%d'%data.shape[0])

In [ ]:
df.head()

In [ ]:
data.shape

In [ ]:
# 贷款基本情况：
# 贷款品质中以好账为主，占比高达92.43%，坏账占比不到8%，情况乐观；
# Lendding Club 平台坏账比例在2016-2018年明显下降，说明平台开始重视对风险的控制；
# 用户画像：
# 用户主要分布在加州，因为Lending Club总部在加州，对本地业务开拓比较深；其次是纽约州和德克萨斯州；
# 用户职业主要都是老师、管理者；
# 大部分用户工作了10+年以上，其余用户工作年限非常均匀，从1到9年的都有，数量相差不多;
# 极大部分用户年收入都大于20000美元，其中高于60000的用户占比大于50%；
# 大部分用户贷款是为了债务整合(借新债还旧债)、还信用卡；50% 以上用户房子还在按揭，40%用户还在租房，不到10%的用户拥有自己的房子
# 绝大部分用户的 贬损公共设施的记录次数小于三次

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
#贷款品质情况
data['贷款现状'].value_counts().plot.pie(
    autopct='%1.2f%%', ax=ax1, fontsize=12, startangle=70)
ax1.set_title('GOOD OR BAD')
ax1.set_ylabel("贷款现状（%）")
ax1.legend()
#正负样本数量差距悬殊，对于后面建模而言是个很大问题

#发放贷款数量按照年度分布情况
sns.barplot(
    x='贷款获得资助的年份',
    y='贷款金额',
    data=data,
    hue='贷款现状',
    estimator=lambda x: len(x) / len(data) * 100,
    ax=ax2)
ax2.set_title('贷款金额')
ax2.set_ylabel('%')
ax2.set_xlabel('贷款获得资助的年份')
ax2.legend()
# plt.savefig("pictures/Lending Club 贷款质量与总额分布情况.png",dpi=1000,bbox_inches = 'tight')
plt.savefig(os.getcwd()+'/datasets/src/'+"temp/pictures/Lending Club 贷款质量与总额分布情况.png",dpi=1000,bbox_inches = 'tight')

In [ ]:
# 地域分布
fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
df['最后一次借款_州简称'].value_counts()[0:19].plot(kind='bar',color="royalblue",fontsize=12)
plt.title("Lending Club 客户地域分布情况",fontsize=16)
plt.subplots_adjust(top=1,bottom=0,left=0,right=1,hspace=0,wspace=0)
#plt.savefig("temp/pictures/Lending Club 客户地域分布情况.png",dpi=1000,bbox_inches = 'tight')
plt.show()

In [ ]:
#就业年限
fig2=plt.figure()
ax1 = fig2.add_subplot(111)
df['最后一次借款_就业年限'].value_counts().plot(kind='bar',color="royalblue",fontsize=12)
plt.title("Lending Club 客户工作年龄分布情况",fontsize=16)
plt.subplots_adjust(top=1,bottom=0,left=0,right=1,hspace=0,wspace=0)
#plt.savefig("temp/pictures/Lending Club 客户工作年龄分布情况.png",dpi=1000,bbox_inches = 'tight')
plt.show()

In [ ]:
# 职业分布
fig3 = plt.figure()
ax1 = fig3.add_subplot(111)
df['最后一次借款_职称'].value_counts()[0:14][::-1].plot(kind='barh',color="royalblue",fontsize=12)
plt.title("Lending Club 客户职业分布情况",fontsize=16)
plt.subplots_adjust(top=1,bottom=0,left=0,right=1,hspace=0,wspace=0)
#plt.savefig("temp/pictures/Lending Club 客户职业分布情况.png",dpi=1000,bbox_inches = 'tight')#解决图片不清晰，不完整的问题
plt.show()

In [ ]:
fig4=plt.figure()
ax1 = fig4.add_subplot(111)
df['最后一次借款_年收入层级'].value_counts()[::-1].plot(kind='barh',color="royalblue",fontsize=12)
plt.title("Lending Club 客户年收入($)分布情况",fontsize=16)
plt.subplots_adjust(top=1,bottom=0,left=0,right=1,hspace=0,wspace=0)
#plt.savefig("temp/pictures/Lending Club 客户年收入($)分布情况.png",dpi=1000,bbox_inches = 'tight')
plt.show()

In [ ]:
fig5=plt.figure()
ax1 = fig5.add_subplot(111)
df['最后一次借款_房屋所有权状态'].replace({"MORTGAGE":"还贷","RENT":"租房","OWN":"自有住房","ANY":"其他","OTHER":"其他","NONE":"其他"},inplace=True)
plt.pie(df['最后一次借款_房屋所有权状态'].dropna().value_counts(),labels=["还贷","租房","自有住房","其他"],autopct="%1.1f%%")
plt.title("Lending Club 客户住房分布情况",fontsize=16)
plt.subplots_adjust(top=1,bottom=0,left=0,right=1,hspace=0,wspace=0)
#plt.savefig("temp/pictures/Lending Club 客户住房分布情况.png",dpi=1000,bbox_inches = 'tight')
plt.show()

In [ ]:
fig6=plt.figure()
ax1 = fig6.add_subplot(111)
pub_v=df['最后一次借款_贬损公共记录的数量'].value_counts()
x_ticks=list(range(6))
pub_v[0:5].plot(kind='bar',color='royalblue',fontsize=12)
plt.xticks(x_ticks,rotation=0)
pub_p=pub_v/pub_v.sum()     # 计算所占百分比
pub_p=pub_p.apply(lambda x:format(x,'.2%'))[0:5] # 转换百分比显示
plt.title("1.6 Lending Club 贬损公共记录分布情况",fontsize=16)
plt.subplots_adjust(top=1,bottom=0,left=0,right=1,hspace=0,wspace=0)
#plt.savefig("temp/pictures/Lending Club 贬损公共记录分布情况.png",dpi=1000,bbox_inches = 'tight') # 解决图片不清晰，不完整的问题
plt.show()

In [ ]:
fig7=plt.figure()
ax1=fig7.add_subplot(111)
new_ticks=np.linspace(-2,48,6)
plt.hist(df['最后一次借款_收入负债比（总额）'].dropna(),range=(-2,50),bins=10,color="royalblue")
plt.xticks(new_ticks)
plt.title("Lending Club 收入负债比（总额）分布情况",fontsize=16)
plt.subplots_adjust(top=1,bottom=0,left=0,right=1,hspace=0,wspace=0)
#plt.savefig("temp/pictures/Lending Club 收入负债比（总额）分布情况.png",dpi=1000,bbox_inches = 'tight')
plt.show()

In [ ]:
fig8 = plt.figure()
ax1 = fig8.add_subplot(111)
df['最后一次借款_贷款目的'].value_counts(ascending=True).plot(kind='barh',fontsize=12,color="royalblue")
plt.title("Lending Club 贷款目的分布情况",fontsize=16)
plt.subplots_adjust(top=1,bottom=0,left=0,right=1,hspace=0,wspace=0)
#plt.savefig("temp/pictures/Lending Club 贷款目的分布情况.png",dpi=1000,bbox_inches = 'tight')
plt.show()

In [ ]:
#标签体系设计
#简单的来说就是可以把用户分配到多少个类里面去，当然每个用户也都可以分到多个不同的类上面
#三种思路：1.结构化标签体系，2.半结构化标签体系，3.非结构化标签体系
#1.结构化：可以组织成比较规整的树或者森林，明确的层级划分和父子关系
#2.半结构化：某种程度上体现了结构化，但是不规整
#3.非结构化：各个标签就事论事，没有体系而言。举例：用户的搜索关键词
#本案例是结构化数据

In [ ]:
indi_attr = ['最后一次借款_职称',  '最后一次借款_就业年限', '最后一次借款_房屋所有权状态', '最后一次借款_年收入', '最后一次借款_邮政编码', '最后一次借款_州简称']
df[indi_attr].head()

In [ ]:
df['最后一次借款_职称'].head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
vectorizer = CountVectorizer(min_df=1000)#设定最小的词频为10000
count = vectorizer.fit_transform(df['最后一次借款_职称'].fillna(''))#词袋计数
transformer = TfidfTransformer()
tfidf_matrix = transformer.fit_transform(count)

df_job= pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names(), index=df.index)
df_job.head()

In [ ]:
print(count[:5])

In [ ]:
print(tfidf_matrix[:5])

In [ ]:
df_job.columns

In [ ]:
fact_label = data.groupby('用户ID')['贷款金额'].agg([np.sum, np.mean, np.min, np.max, np.count_nonzero])#聚合窗口
fact_label.columns = ['历史_贷款金额总和', '历史_贷款平均金额', '历史_贷款最小金额', '历史_贷款最大金额', '历史_贷款次数']
fact_label = fact_label.reset_index()
fact_label.head()

In [ ]:
#因子分析，先两两配对，计算相关性、
# 我们先使用探索性因子分析的方法来找出标签数据中含有的因子个数，以及各个因子和各个标签之间的相关程度，试图揭示一套相对比较大的标签数据集的内在结构。
# 两两相关性高于阈值的标签暂留一个
# 原因是进行因子分析时需进行矩阵分解，如果标签之间相关性太高可能造成求矩阵的伪逆时失败

#cor_stat = df.select_dtypes(exclude=object) #筛选出来需要用的数据
#cor_stat = df.select_dtypes(exclude=object).corr(method='spearman') #建立corr矩阵
#cor_stat = df.select_dtypes(exclude=object).corr(method='spearman').stack() #从矩阵改变成结构目录
cor_stat = df.select_dtypes(exclude=object).corr(method='spearman').stack().reset_index()#index从树状结构变成表结构
cor_stat.columns = ['feat1', 'feat2', 'r']
cor_stat = cor_stat[cor_stat['feat1']!=cor_stat['feat2']]
cor_stat = cor_stat.sort_values(by='r', ascending=False)
cor_stat.head(20)

In [ ]:
drop_col=[]
while cor_stat[cor_stat['r']>0.95].shape[0]!=0:#筛选corr高于0.95的值
    nomi = cor_stat.iloc[0]['feat1']
    drop_col += [nomi]
    print('相关性高于阈值的标签暂留一个，删去：', nomi)#每次只删第一列
    cor_stat = cor_stat[(cor_stat['feat1']!=nomi)&(cor_stat['feat2']!=nomi)]

In [ ]:
std_stat = df.std()#计算每一列数据的标准差
std_stat[std_stat<100].hist()

In [ ]:
std_stat[std_stat<0.5].shape

In [ ]:
weak_std_col = std_stat[std_stat<0.5].index.tolist()#有50个小于0.5标准差的
weak_std_col

In [ ]:
#只留下可能与潜在标签有业务关系的特征
df2=df.select_dtypes(exclude=object).drop(['ID','用户ID','最后一次借款_贷款获得资助的月份'] + weak_std_col + drop_col, axis=1)
print('用于因子分析的特征有%d个'%df2.shape[1])

In [ ]:
#标准化数据
ss = StandardScaler()
df2 = pd.DataFrame(ss.fit_transform(df2), columns=df2.columns)
# 对特征进行缺失值填补
imp = SimpleImputer()
df2 = pd.DataFrame(imp.fit_transform(df2), columns=df2.columns)

In [ ]:
# 充分性测试adequency test (Bartlett's Test)
chi_square_value,p_value=calculate_bartlett_sphericity(df2)
print(chi_square_value)
print(p_value)
# Bartlett's test of sphericity 是用来检测观察到的变量之间是否关联, 如果检测结果在统计学上不显著, 就不能采用因子分析.
# 此处我们的p-value=0, 表明观察到的相关矩阵不是一个identity matrix.

In [ ]:
# 充分性测试（Kaiser-Meyer-Olkin Test）
kmo_all,kmo_model=calculate_kmo(df2)
# print(kmo_all)
print(kmo_model)
#Kaiser-Meyer-Olkin (KMO) Test 就是需要kmo值要大于0.6才能进行下一步因子分析

In [ ]:
fa = FactorAnalyzer(25, rotation=None)
fa.fit(df2)
ev, v = fa.get_eigenvalues()
plt.scatter(range(1,df2.shape[1]+1),ev)
plt.plot(range(1,df2.shape[1]+1),ev)
plt.title('Scree Plot')
plt.xlabel('Factors')
plt.ylabel('Eigenvalue')
plt.grid()
plt.show()

In [ ]:
#选7个变量
fa = FactorAnalyzer(6, rotation="varimax")
fa.fit(df2)
# 系数
fa.loadings_[:5]

In [ ]:
df_cm = pd.DataFrame(fa.loadings_, index=df2.columns)
plt.figure(figsize = (12,18))
sns.heatmap(df_cm, annot=True, cmap="BuPu")
# 设置y轴的字体的大小
plt.title('Factor Analysis', fontsize='xx-large')
# Set y-axis label
plt.ylabel('Sepal Width', fontsize='xx-large')
# plt.savefig('temp/pictures/factorAnalysis.png', dpi=500)

In [ ]:
# 第一个因子体现的是借款人信用额度的维度。
# 第二个因子主要解释了借款人共债风险的维度
# 第三个因子主要解释了借款人资金紧张情况的维度
# 第四个因子主要解释了借款产品相关的维度
# 第五个因子体现了借款人还款情况的维度

In [ ]:
# 因子分析得出的只是数据集内部在统计学上面的联系，我们还需要结合对各个标签分析的结果，
# 及业务逻辑上的考量来选定用来合成标签所用的特征。
# 如果有待合成标签的人工标签，我们可以通过逻辑回归建模得到特征与待合成标签的正负关联性和特征重要性。
# 但如果没有待合成标签的实际标签，也不能获取能够令人信服的实际标签替代品时，
# 我们可以通过因子分析判断特征的重要性，并且通过业务逻辑判断特征与待合成标签相关性的正负性。
from sklearn.preprocessing import StandardScaler
def mix_index(data, cols, neg_col):
    # 融合方式：标准化后取平均
    stdsc=StandardScaler()
    dat = data[cols]
    dat[neg_col] = -dat[neg_col]
    for col in cols:
        data[col] = stdsc.fit_transform(data[[col]]) 
        data[col].fillna(data[col].mean(), inplace=True)
    return data[cols].apply(lambda x: x.sum()/len(cols), axis=1)

In [ ]:
#借款人信用风险指数
#我们列出了对因子的贡献度在0.4以上的特征
col1 = df_cm[0][df_cm[0]>0.4].index.tolist()
col1

In [ ]:
#结合之前的逻辑需求
col1 = ['最后一次借款_承诺放贷金额（贷款人）',
       '最后一次借款_年收入',
       '最后一次借款_未结信用额度',
       '最后一次借款_总信贷周转余额',
       '最后一次借款_当前信用额度的总数',
       '最后一次借款_所有帐户的当前总余额',
       '最后一次借款_所有分期付款帐户的当前总余额',
       '最后一次借款_所有周转账户的最大当前余额',
       '最后一次借款_总循环高信用/信用额度',
       '最后一次借款_除房贷之外的信用负债']
col1

In [ ]:
neg_col1 = ['最后一次借款_承诺放贷金额（贷款人）', '最后一次借款_年收入', '最后一次借款_总信贷周转余额',
           '最后一次借款_当前信用额度的总数', '最后一次借款_所有帐户的当前总余额', '最后一次借款_所有分期付款帐户的当前总余额',
           '最后一次借款_所有周转账户的最大当前余额', '最后一次借款_总循环高信用/信用额度']
df.loc[:, '借款人信用风险指数'] = mix_index(df, col1, neg_col1)

col2 = df_cm[1][df_cm[1]>0.4].index.tolist()
col2

col2 = ['最后一次借款_过去12个月的信用查询次数',
        '最后一次借款_过去12个月内开立的分期付款帐户数量',
        '最后一次借款_过去24个月内开立的分期付款帐户数量',
        '最后一次借款_所有分期付款帐户的当前总余额',
        '最后一次借款_个人理财咨询数量',
        '最后一次借款_除房贷之外的信用负债']
col2

In [ ]:
neg_col2 = ['最后一次借款_所有分期付款帐户的当前总余额']
df.loc[:, '借款人共债风险指数'] = mix_index(df, col2, neg_col2)

In [ ]:
col3 = df_cm[2][df_cm[2]>0.4].index.tolist()
col3

neg_col3 = []
df.loc[:, '借款人资金周转风险指数'] = mix_index(df, col3, neg_col3)

In [ ]:
col4 = df_cm[4][df_cm[4]>0.4].index.tolist()
col4

col4 = ['最后一次借款_迄今已收本金', '最后一次借款_上次收到的总付款金额',
        '最后一次借款_剩余未偿还本金', '最后一次借款_剩余未偿还本金（本人）']
col4

neg_col4 = ['最后一次借款_剩余未偿还本金', '最后一次借款_剩余未偿还本金（本人）']
df.loc[:, '借款人还款意愿指数'] = mix_index(df, col4, neg_col4)

df[['用户ID', '借款人信用风险指数', '借款人共债风险指数', '借款人资金周转风险指数', '借款人还款意愿指数']]
#.to_csv('./work/member_tag.csv', index=None)

In [ ]:
#机器学习部分
df[indi_attr].isnull().sum()/len(df)#查看缺失率等情况

In [ ]:
imp_model_col = \
['最后一次借款_上次公开记录以来的月数',
 '最后一次借款_剩余未偿还本金',
 '最后一次借款_剩余未偿还本金（本人）',
 '最后一次借款_年收入',
 '最后一次借款_年收入层级_High',
 '最后一次借款_年收入层级_Low',
 '最后一次借款_年收入层级_Medium',
 '最后一次借款_当前信用额度的总数',
 '最后一次借款_循环利用率',
 '最后一次借款_总信贷周转余额',
 '最后一次借款_房屋所有权状态等级',
 '最后一次借款_承诺放贷金额（贷款人）',
 '最后一次借款_承诺放贷金额（贷款）',
 '最后一次借款_收入负债比（总额）',
 '最后一次借款_收入验证',
 '最后一次借款_是否实施还款计划_n',
 '最后一次借款_是否实施还款计划_y',
 '最后一次借款_最后一次拖欠以来的月数',
 '最后一次借款_未结信用额度',
 '最后一次借款_每月还款',
 '最后一次借款_贬损公共记录的数量',
 '最后一次借款_贷款子等级',
 '最后一次借款_贷款期数',
 '最后一次借款_贷款等级_A',
 '最后一次借款_贷款等级_B',
 '最后一次借款_贷款等级_C',
 '最后一次借款_贷款等级_D',
 '最后一次借款_贷款等级_E',
 '最后一次借款_贷款等级_F',
 '最后一次借款_贷款等级_G',
 '最后一次借款_贷款等级_INT',
 '最后一次借款_贷款金额',
 '最后一次借款_过去12个月的信用查询次数',
 '最后一次借款_过去6个月的查询数目',
 '最后一次借款_过去两年借款人逾期30天以上的事件数']

In [ ]:
train = df[imp_model_col + ['ID', '最后一次借款_就业年限']].copy()
train.head()

In [ ]:
test = train[train['最后一次借款_就业年限'].isnull()]
train = train[train['最后一次借款_就业年限'].notnull()]
train, val = train_test_split(train, test_size=0.2, random_state=42)

In [ ]:
#y是最后一次借款_就业年限
dtrain = xgb.DMatrix(train.drop(['ID', '最后一次借款_就业年限'], axis=1), label=train['最后一次借款_就业年限'])
dval = xgb.DMatrix(val.drop(['ID', '最后一次借款_就业年限'], axis=1), label=val['最后一次借款_就业年限'])
dtest = xgb.DMatrix(test.drop(['ID', '最后一次借款_就业年限'], axis=1))

In [ ]:
train.head(5)

In [ ]:
#定义一些超参数
params = {}
params['objective'] = 'reg:linear'
params['eval_metric'] = 'rmse'
params['min_child_weight'] = 10
params['eta'] = 0.1
params['max_depth'] = 4
params['subsample'] = 0.7
params['colsample_bytree'] = 0.5
params['silent'] = 0
params['seed'] = 42
params['lambda'] = 10
params['max_delta_step'] = 0
params['early_stopping_rounds'] = 10

watchlist = [(dtrain, 'train'), (dval, 'eval')]
model1 = xgb.train(params, dtrain, num_boost_round=10000, evals=watchlist,
                  verbose_eval=True, early_stopping_rounds=30)

In [ ]:
pred_test = model1.predict(dtest)
test['最后一次借款_就业年限'] = pred_test

df.set_index('ID', inplace=True)
df.loc[test['ID'], '最后一次借款_就业年限'] = test['最后一次借款_就业年限']
df = df.reset_index()

df['最后一次借款_就业年限'].isnull().sum()

In [ ]:
after_loan_feature = ['最后一次借款_上次收到的总付款金额', '最后一次借款_迄今已收滞纳金', '最后一次借款_剩余未偿还本金（本人）',
                      '最后一次借款_剩余未偿还本金', '最后一次借款_迄今已收本金', '最后一次借款_贷款子等级', '借款人信用风险指数',
                      '借款人共债风险指数', '借款人资金周转风险指数', '借款人还款意愿指数', '最后一次借款_贷款现状_Bad Loan',
                      '最后一次借款_迄今已收利息', '最后一次借款_收取总回收费用', '最后一次借款_迄今还款总额',
                      '最后一次借款_迄今为止收到的由投资者资助的部分总金额', '最后一次借款_贷款现状_Good Loan'] + \
                     [i for i in list(df) if '贷款等级' in i]

y = df['最后一次借款_贷款现状标签']
x = df.drop(['ID', '用户ID', '最后一次借款_贷款获得资助的年份','最后一次借款_贷款获得资助的月份', '最后一次借款_贷款现状标签'] + \
            after_loan_feature, axis=1).select_dtypes(exclude=object)

x_train, x_test, true_train, true_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train, x_val, true_train, true_val = train_test_split(x_train, true_train, test_size=0.2, random_state=42)
dtrain = xgb.DMatrix(x_train, label=true_train)
dval = xgb.DMatrix(x_val, label=true_val)
dtest = xgb.DMatrix(x_test, label=true_test)

y.value_counts()

In [ ]:
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'auc'
params['min_child_weight'] = 11
params['eta'] = 0.1
params['max_depth'] = 2
params['subsample'] = 0.5
params['colsample_bytree'] = 0.5
params['silent'] = 0
params['seed'] = 2019
params['lambda'] = 10
params['max_delta_step'] = 0
params['scale_pos_weight'] = 12
params['early_stopping_rounds'] = 10

watchlist = [(dtrain, 'train'), (dval, 'eval')]
model1 = xgb.train(params, dtrain, num_boost_round=10000, evals=watchlist,
                  verbose_eval=True, early_stopping_rounds=30)

In [ ]:
# #!pip install model_evaluation
# from model_evaluation import get_xgb_fi
# fi,_ = get_xgb_fi(model1,'rank')
# fi.head(5)